# 네이버 검색결과 이메일로 발송.

1. 전체 검색 결과를 구분별로 가져오기
2. 세부 파트별로 가져오기 (블로그, 뉴스, 카페 글)

## 1. 전체 검색 결과를 구분별로 가져오기
### 설계
1. 브라우저 설정 
2. target_url(https://search.naver.com/search.naver?query={검색내용}) 설정 및 접속
3. HTML 분석 & 내용을 형식별로 구분 (카페, 블로그, 지식In 등)
4. 가져올 내용들을 정리 (작성일자, 이미지, description, 제목, 링크, <블로그 이름>, 링크 url 등)
5. 시각화 (엑셀, or html) 등

#### 1. 브라우저 설정

In [8]:
# 1. 브라우저 설정
"""
정말 많은 brower가 있습니다만,
본 예제에서는 OS에 독립적으로 실행되길 원하므로,
보편적인 Chrome을 사용하겠습니다.
만약 chrome driver가 없다면 아래 링크에서 설치하십시요.
- http://chromedriver.storage.googleapis.com/index.html
"""

from selenium import webdriver

# TODO: Option에 대한 설명 필요.
# options = None
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": '.'
})
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.implicitly_wait(10)


### 아래와 같은 많은 브라우저들이 있습니다.

# driver = webdriver.FireFox
# webdriver.FirefoxProfile

# driver = webdriver.Ie
# driver = webdriver.Opera

# driver = webdriver.PhantomJS

# driver = webdriver.Remote
# webdriver.DesiredCapabilities

# driver = webdriver.safairi

## ...


#### 2. target_url(https://search.naver.com/search.naver?query={검색내용}) 설정 및 접속

In [9]:
base_url = 'https://search.naver.com/search.naver?query='
search_content = '마약베개'

target_url = base_url + search_content

driver.get(target_url)

#### 3. HTML 분석 (Top-Down 방식) & 내용을 형식별로 구분 (카페, 블로그, 지식In 등)

In [15]:
## 검색 결과가 나오는 컨텐츠 
## <div id="main_pack" class="main_pack">...</div>
main_pack = driver.find_element_by_xpath('//*[@id="main_pack"]')
# print(main_pack)
# print(dir(main_pack))

# 방법1. 
nx_related_keyword = main_pack.find_element_by_id('nx_related_keywords')
nx_brand_search_section = main_pack.find_element_by_class_name('brand_search')
nx_powerlink_section = main_pack.find_element_by_class_name('_pl_section')
nx_shopping_section = main_pack.find_element_by_id('_shopping_guide_view')
nx_blog_section = main_pack.find_element_by_class_name('_prs_blg')
nx_cafe_section = main_pack.find_element_by_class_name('_prs_caf')
nx_post_section = main_pack.find_element_by_class_name('sp_post')
nx_website_section = main_pack.find_element_by_class_name('sp_website')
nx_biz_site_section = main_pack.find_element_by_class_name('_bz_section')




#### 4. 가져올 내용들 스크린샷

In [51]:
nx_related_keyword = main_pack.find_element_by_id('nx_related_keywords')
nx_brand_search_section = main_pack.find_element_by_class_name('brand_search')
nx_powerlink_section = main_pack.find_element_by_class_name('_pl_section')
nx_shopping_section = main_pack.find_element_by_id('_shopping_guide_view')
nx_blog_section = main_pack.find_element_by_class_name('_prs_blg')
nx_cafe_section = main_pack.find_element_by_class_name('_prs_caf')
nx_post_section = main_pack.find_element_by_class_name('sp_post')
nx_website_section = main_pack.find_element_by_class_name('sp_website')
nx_biz_site_section = main_pack.find_element_by_class_name('_bz_section')

## 스크린샷을 다음처럼 파일로 저장할 수 있습니다.
nx_related_keyword.screenshot('nx_related_keyword.png')

## 하지만 우린 스크린샷을 이용해 메일을 보낼 것이기 때문에,
## 파일로 저장하지 않고, 이미지를 변수에 저장해보도록 합니다. 
## 방법은 다음처럼 screenshot_as_base64 을 이용합니다.
screenshots = {
    'related_keyword': nx_related_keyword.screenshot_as_base64,
    'brand_search_section': nx_brand_search_section.screenshot_as_base64,
    'powerlink_section': nx_powerlink_section.screenshot_as_base64,
    'shopping_section':nx_shopping_section.screenshot_as_base64,
    'blog_section':nx_blog_section.screenshot_as_base64,
    'cafe_section':nx_cafe_section.screenshot_as_base64,
    'post_section':nx_post_section.screenshot_as_base64,
    'website_section':nx_website_section.screenshot_as_base64,
    'biz_site_section':nx_biz_site_section.screenshot_as_base64
}



#### 5. 스크린샷 이미지 태그로 만들기

In [49]:
img_tag_screenshots = {} # img_tag들을 저장할 스크린샷 dictionary

for s_shot in screenshots:
    # 보기 좋게 최대 사이즈도 정해 놓읍시다.
    img_tag_screenshots[s_shot] = f'<img style="max-height=600px; max-width:800px;" src="data:image/png;base64, {screenshots[s_shot]}" />'    


#### 6. HTML 이미지 태그 메일로 보내기

In [55]:
import smtplib, ssl
from email.mime.text import MIMEText


port = 465  # For SSL
smtp_server = "smtp.naver.com"
sender_email = "<your email>"  # 제가 저에게 보냅니다.
receiver_email = "<your email>"  # 제가 저에게 보냅니다.
password = "<your password>"

subject= f"[정기수신] {search_content}의 검색결과" 

html_message = f"""\
# {subject}
<br/>
# {search_content}에 따른 검색결과는 다음과 같습니다.
"""

## for문을 이용해 편하게 뽑아내봅시다.
for tag_key in img_tag_screenshots:
    html_message += "<br/><br/>" # 공백 두개 추가합니다.
    html_message += f'<h1>{tag_key}</h1> <br/>'
    html_message += f'{img_tag_screenshots[tag_key]}' 


message = MIMEText(html_message, 'html', _charset="UTF-8")
message['Subject'] = subject
message['From'] = sender_email
message['to'] = receiver_email

context = ssl.create_default_context()
with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message.as_string())

#### 가져올 내용 정리

In [ ]:

## 연관검색어 형식 구분
# keyword 별로 나열.
keywords = []
# print(dir(related_keywords))

keywords_tag = related_keywords.find_elements_by_tag_name('li')
for x in keywords_tag:
    keywords.append({
        'text':x.text,
        'link':x.find_element_by_tag_name('a').get_attribute('href')
    })
# pprint(keywords)

## 브랜드 서치 섹션 형식 구분
brand_search_section.screenshot('brand_search_section.png') # 해당부분만 스크린샷 찍기

## 지식쇼핑 섹션
sp_shop_section.screenshot('sp_shop.png')
print("C")

## 블로그 섹션
blog_list = []
blog_contents = blog_section.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
for blog_content in blog_contents:
    title = blog_content.find_element_by_class_name('sh_blog_title')
    blog_list.append({
        'image':blog_content.find_element_by_class_name('thumb').find_element_by_tag_name('img').get_attribute('src'),
        'link':title.get_attribute('href'),
        'title':title.text,
        'desc':blog_content.find_element_by_class_name('sh_blog_passage').text
    })
# pprint(blog_list)
print("Hi")

## cafe 섹션
# 자세히 보면 블로그와 구조가 똑같아보여요!
cafe_list = []
cafe_contents = cafe_section.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
for cafe_content in cafe_contents:
    title = cafe_content.find_element_by_class_name('sh_cafe_title')
    cafe_list.append({
        'image':cafe_content.find_element_by_class_name('thumb').find_element_by_tag_name('img').get_attribute('src'),
        'link':title.get_attribute('href'),
        'title':title.text,
        'desc':cafe_content.find_element_by_class_name('sh_cafe_passage').text
    })
pprint(cafe_list)
print("HO")

In [ ]:
## 1. 전체 검색 결과를 구분별로 가져오기
### 설계
1. 브라우저 설정 
2. target_url(https://search.naver.com/search.naver?query={검색내용}) 설정 및 접속
3. HTML 분석 & 내용을 형식별로 구분 (카페, 블로그, 지식In 등)
4. 가져올 내용들을 정리 (작성일자, 이미지, description, 제목, 링크, <블로그 이름>, 링크 url 등)
5. 시각화 (엑셀, or html) 등